## Solveur SAT

Télécharger glophersat.zip sur Edunao et le déziper.

In [1]:
#Set 
N = 3 #nombre de matière
s = {x for x in range(N)}

In [2]:
#Variable aplha
alpha = []
for i in range(0,N):
    for k in range(0,21):
        alpha.append((i,k)) # ==> donne tous les alpha i k 

#Dictionnaire alpha
v2i_alpha = {v : i+1 for i,v in enumerate(alpha)} # à chaque variable associe un nombre
A = len(v2i_alpha)


In [3]:
from itertools import chain
from itertools import combinations

#Créer l'ensemble des subset 
def powerset(iterable): 
    s = list(iterable)
    return( chain.from_iterable(combinations(s, r) for r in range(len(s)+1)) )

#L'ensemble des subset 
subsets = list(powerset(s)) 
print('subsets')
print(subsets)

#Dictionnaire beta
v2i_beta = {v : A+i+1 for i,v in enumerate(subsets)}
print('v2i_beta')
print(v2i_beta)

subsets
[(), (0,), (1,), (2,), (0, 1), (0, 2), (1, 2), (0, 1, 2)]
v2i_beta
{(): 64, (0,): 65, (1,): 66, (2,): 67, (0, 1): 68, (0, 2): 69, (1, 2): 70, (0, 1, 2): 71}


In [4]:
#Clause 1
clause_1 = []

for i in range(0,N):
    for k in range(0,21):
        for j in range(k+1,20):
            clause_1.append([-v2i_alpha[(i,k)], v2i_alpha[(i,j)]])

#print(clause_1)
         

In [5]:
#Clause 2
clause_2 = []

for i in subsets:
    C_prime = set(i)
    for j in subsets:
        C = set(j)
        if C.issubset(C_prime):
            clause_2.append((-v2i_beta[j], v2i_beta[i]))

#print(clause_2)

In [6]:
#Clause 3

clause_3 = []

for c in subsets:
    C = set(c)
    D = s.difference(C)
    j = tuple(D)
    alpha = []
    for k in range(0,21):
        for i in range(0,N):
            if i in C:
                alpha.append(v2i_alpha[(i,k)])
    
    clause_3.append((alpha,v2i_beta[j]))
            
#print(clause_3)

In [7]:
clause_4 = []

for c in subsets:
    C = set(c)
    j = tuple(C)
    alpha = []
    for k in range(0,21):
        for i in range(0,N):
            if i in C:
                alpha.append(-v2i_alpha[(i,k)])
    
    clause_4.append((alpha,-v2i_beta[j]))

#print(clause_4)

In [14]:
#Construction du DIMCS et Résolution
i2v = {}

for i in range(len(v2i_alpha)):
    i2v[i+1] = list(v2i_alpha.keys())[list(v2i_alpha.values()).index(i+1)]

for i in range(len(v2i_beta)):
    i2v[i+1+A] = list(v2i_beta.keys())[list(v2i_beta.values()).index(i+1+A)]

import subprocess

def clauses_to_dimacs(clauses,numvar) :
    dimacs = 'c This is it\np cnf '+str(numvar)+' '+str(len(clauses))+'\n'
    for clause in clauses :
        for atom in clause :
            dimacs += str(atom) +' '
        dimacs += '0\n'
    return dimacs

def write_dimacs_file(dimacs, filename):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)

#Attention à utiliser la vesion du solveur compatible avec votre système d'exploitation, mettre le solveur dans le même dossier que ce notebook        

def exec_gophersat(filename, cmd = "./gophersat.exe", encoding = "utf8") :
    result = subprocess.run([cmd, filename], stdout=subprocess.PIPE, check=True, encoding=encoding)
    string = str(result.stdout)
    lines = string.splitlines()

    if lines[1] != "s SATISFIABLE":
        return False, [], {}

    model = lines[2][2:].split(" ")

    return True, [int(x) for x in model if int(x) != 0], { i2v[abs(int(v))] : int(v) > 0 for v in model if int(v)!=0} 


In [16]:
#Lancer la résolution

myClauses= clause_1 + clause_2 + clause_3 + clause_4
myDimacs = clauses_to_dimacs(myClauses,len(alpha))

write_dimacs_file(myDimacs,"./workingfile.cnf")
res = exec_gophersat("./workingfile.cnf")

#Résultat
print(res)

FileNotFoundError: [WinError 2] Le fichier spécifié est introuvable